# 네이버 블로그 크롤링

In [5]:
import requests
from bs4 import BeautifulSoup
#from selenium import webdriver

In [6]:
import time
import random
import datetime
import re
#import pickle

## blog URL

In [67]:
def get_blogs(url):
    results = []
    urls = []
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    lis = soup.find_all('li', attrs={'class':'sh_blog_top', 'id':re.compile('sp_blog')})
    
    for li in lis:

        title = li.find('a', attrs={'class':re.compile(r'_sp_each_title')}).get('title').strip()
        if li.find('a', attrs={'class':re.compile(r'_sp_each_url')}).text:
            naver_url = li.find('a', attrs={'class':re.compile(r'_sp_each_url')}).get('href').strip()
#        pub = li.find('span', attrs={'class':'_sp_each_source'}).text.strip()
#        date = li.find('span', attrs={'class':'bar'}).next_sibling.strip()
        results.append([title, naver_url])
        urls.append(naver_url)
        print(title, naver_url)
        
    return urls

## real data URL

In [91]:
def get_realdata_url_naverblog(url):
    """
    네이버 블로그 포스트의 실제 데이터가 들어있는 URL을 반환해줌.
    보통 3가지 경우가 있는 것 같음.
    <지금까지 찾은 경우의 수>
    1. id='screenFrame'이 담긴 페이지에서 얻은 URL -> (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    2. (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    3. 검색해서 처음 얻은 URL이 실제 데이터가 들어가 있는 URL임
    """
    base_url = "http://blog.naver.com"
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    
    if soup.find(id='screenFrame'):
        # id = 'screenFrame'이 존재하는 경우 (id: screenFrame URL에서 -> id: mainFrame URL로 넘김)
        url = soup.find(id='screenFrame').get('src')
        r = requests.get(url)
        soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    if soup.find(id='mainFrame'):
        # id = 'mainFrame'이 존재하는 경우 (id: mainFrame URL에서 -> id: real data URL로 넘김)
        url = base_url + soup.find(id='mainFrame').get('src')
    return url   

## Title

In [9]:
def get_title_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 제목을 반환해줌.
    """
    try:
        #블로그 포스트 제목 추출
        title = soup.find('meta', attrs={'property':'og:title'}).get('content')
    except:
        title = 'NA'
    return title

## Datetime

In [10]:
def det_publish_datetime_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 작성일을 반환해줌.
    """
    try:
        #블로그 작성일 추출
        date_tag = soup.find('span', attrs={'class':'se_publishDate pcol2'})
        if date_tag:
            # <span> Tag의 se_publishDate pcol2 클래스 이름으로 작성일 추출이 가능한 경우
            date_str = date_tag.get_text()
        
        else:
            # <p> Tag의 date fil5 pcol2 _postAddDate 클래스 이름으로 작성일 추출이 가능한 경우
            date_tag = soup.find('p', attrs={'class' : 'date fil5 pcol2 _postAddDate'})
            if date_tag:
                date_str = date_tag.get_text()  
        try:
            #블로그 작성시간이 년, 월, 일로 되어있을 경우
            publish_datetime = datetime.datetime.strptime(date_str, '%Y. %m. %d. %H:%M')
        except:
            #블로그 작성시간이 N시간 전, 방금전 이렇게 표시될 경우
            publish_datetime = str(datetime.datetime.now()).split('.')[0] + ' ' + date_str
    except:
        publish_datetime = 'NA'
    return publish_datetime

## Text

In [11]:
def get_texts_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 본문 내용을 반환해줌.
    """
    try:
        #블로그 본문 추출
        #보통은 <span> Tag에, id: 'SE'로 시작하는 곳에 본문이 담겨있음
        temp = soup.findAll("span", attrs= {'id': re.compile('^SE')})
        
        texts = []
        for t in temp:
            temp_text = t.get_text()
            if temp_text != '\u200b':
                texts.append(temp_text)
                    
        if not texts:
            # 블로그의 본문 텍스트가 전혀 추출되지 않았을 경우에 한번 더 시도해본다.
            # <p class="se_textarea"> 내에 존재하는 모든 본문 텍스트를 가져옴
            """
            # <p class="se_textarea"> 내의 <span> Tag에 본문이 들어가 있을 경우 
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                for span in p.findAll("span"):
                    temp_text = span.get_text()
                    if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
            """
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                temp_text = p.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
                        
        if not texts:
            # 위의 두 가지 방법으로 본문 텍스트 추출을 시도했음에도 텍스트가 전혀 추출되지 않을 경우
            # <div> Tag 내의 id 값이 'postViewArea'이고,
            # 다시 그 태그의 내부에 존재하는 <div> Tag들 속에 존재하는 모든 본문 텍스트를 가져옴
            for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div'):
                temp_text = div.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
        
        texts = "\n".join(texts)
    except:
        texts = 'NA'
    return texts

##  naver URL 구성
* where = post(블로그)<br/>
* query = <br/>
* st = sim(관련도순), date(최신순) <br/>
* date_from = <br/>
* date_to = <br/>
* date_option = 0(전체), 2,3,4,5,6,7(1일,1주,1개월,6개월,1년), 8(직접입력) <br/>
* srchby = all(전체), title(제목)<br/>

## selenium으로 blog url 불러오기

In [2]:
from tqdm import tqdm_notebook
import pandas as pd

In [3]:
data_pd = pd.date_range('2017-09-01', '2019-04-30').strftime("%Y%m%d") # 네이버 클로바 출시일- 2017.10
data_pd

Index(['20170901', '20170902', '20170903', '20170904', '20170905', '20170906',
       '20170907', '20170908', '20170909', '20170910',
       ...
       '20190421', '20190422', '20190423', '20190424', '20190425', '20190426',
       '20190427', '20190428', '20190429', '20190430'],
      dtype='object', length=607)

In [55]:
data_pd[400:]

Index(['20181006', '20181007', '20181008', '20181009', '20181010', '20181011',
       '20181012', '20181013', '20181014', '20181015',
       ...
       '20190421', '20190422', '20190423', '20190424', '20190425', '20190426',
       '20190427', '20190428', '20190429', '20190430'],
      dtype='object', length=207)

In [88]:
query = '"네이버%20클로바"%20스피커'

#pickle_name = q+'_'+startDate+'_'+endDate+'.p'
total_url = []

for date in tqdm_notebook(data_pd):
    for page in range(1,100):
        try:
            time.sleep(0.1)
            print(page)

            url = 'https://search.naver.com/search.naver?date_from='+str(date)+'&date_option=8&date_to='+str(date)+'&dup_remove=1&nso=p%3Afrom'+str(date)+'to'+str(date)+'&query='+query+'&where=post&start='+str(page)

            results = get_blogs(url)

            if results[0] in total_url:
                break

            total_url = total_url + results
        except:
            break     

#pickle.dump(total_results.open(pickle_name,'wb'))


1
1
 https://blog.naver.com/bonamy?Redirect=Log&logNo=221087693369
 https://outia.blog.me/221087617932
2
 https://outia.blog.me/221087617932
1
1
1
1
1
 https://byline.network/2017/09/7-3/
2
 https://byline.network/2017/09/7-3/
1
 https://blog.naver.com/withcult?Redirect=Log&logNo=221092527285
2
 https://blog.naver.com/withcult?Redirect=Log&logNo=221092527285
1
1
1
AI 스피커 카카오미니  18일부터 예약판매    한글그림기적 https://blog.naver.com/dndyddka?Redirect=Log&logNo=221094640305
170911. 뉴스 모아보기 : 네이버의 AI와 클라우드 플랫폼, 파파고, 클로바 서비스 https://clay1987.blog.me/221094020655
2
170911. 뉴스 모아보기 : 네이버의 AI와 클라우드 플랫폼, 파파고, 클로바 서비스 https://clay1987.blog.me/221094020655
1
인공지능 스피커 WAVE 파격 프로모션 소식 (9/14 커밍순) https://blog.naver.com/clova_ai?Redirect=Log&logNo=221095185987
2
인공지능 스피커 WAVE 파격 프로모션 소식 (9/14 커밍순) https://blog.naver.com/clova_ai?Redirect=Log&logNo=221095185987
1
1
 https://blog.naver.com/honest72?Redirect=Log&logNo=221097077052
2
 https://blog.naver.com/honest72?Redirect=Log&logNo=221097077052
1
1
 http://blog

소니 4K 핸디캠 FDR-AX700... 4K HDR(HLG)과 초고속 하이브리드 AF를 적용한 플래그십 핸디캠 등장... https://its.tistory.com/entry/%EC%86%8C%EB%8B%88-4K-%ED%95%B8%EB%94%94%EC%BA%A0-FDRAX700-4K-HDRHLG%EA%B3%BC-%EC%B4%88%EA%B3%A0%EC%86%8D-%ED%95%98%EC%9D%B4%EB%B8%8C%EB%A6%AC%EB%93%9C-AF%EB%A5%BC-%EC%A0%81%EC%9A%A9%ED%95%9C-%ED%94%8C%EB%9E%98%EA%B7%B8%EC%8B%AD-%ED%95%B8%EB%94%94%EC%BA%A0-%EB%93%B1%EC%9E%A5
2
 https://blog.naver.com/info_mark?Redirect=Log&logNo=221123122707
소니 4K 핸디캠 FDR-AX700... 4K HDR(HLG)과 초고속 하이브리드 AF를 적용한 플래그십 핸디캠 등장... https://its.tistory.com/entry/%EC%86%8C%EB%8B%88-4K-%ED%95%B8%EB%94%94%EC%BA%A0-FDRAX700-4K-HDRHLG%EA%B3%BC-%EC%B4%88%EA%B3%A0%EC%86%8D-%ED%95%98%EC%9D%B4%EB%B8%8C%EB%A6%AC%EB%93%9C-AF%EB%A5%BC-%EC%A0%81%EC%9A%A9%ED%95%9C-%ED%94%8C%EB%9E%98%EA%B7%B8%EC%8B%AD-%ED%95%B8%EB%94%94%EC%BA%A0-%EB%93%B1%EC%9E%A5
1
클로바 인공지능 스피커 프렌즈 브라운 개봉기 & 디자인 http://atombit.net/221119712203
[인공지능 스피커 프렌즈 리뷰] 깜찍한 브라운이 강림하다~ 네이버 클로바 AI 스피커를 먼저 써보니... https://its.tistory.com/entry/%EC%9D%B8%EA%B3%B5%EC%A7%80%

네이버 프렌즈  인공지능 AI 스피커 배송 도착! 사용기 https://blog.naver.com/sksms1105?Redirect=Log&logNo=221130591061
인공지능 스피커 프렌즈, 네이버 클로바 AI 프렌즈 스피커 사용기 https://cdmanii.com/6252?category=423952
인공지능 스피커 프렌즈 - 개봉기 https://blog.naver.com/clova_ai?Redirect=Log&logNo=221130415225
2
네이버 클로바 AI스피커 프렌즈스피커 샐리 https://blog.naver.com/ryland?Redirect=Log&logNo=221129748920
(사용기)네이버 라인 스피커 https://sufeel97.blog.me/221130038887
언제 어디서든 함께하는 인공지능 스피커 '프렌즈' https://blog.naver.com/al_satang?Redirect=Log&logNo=221130215651
네이버 프렌즈  인공지능 AI 스피커 배송 도착! 사용기 https://blog.naver.com/sksms1105?Redirect=Log&logNo=221130591061
인공지능 스피커 프렌즈, 네이버 클로바 AI 프렌즈 스피커 사용기 https://cdmanii.com/6252?category=423952
인공지능 스피커 프렌즈 - 개봉기 https://blog.naver.com/clova_ai?Redirect=Log&logNo=221130415225
1
인공지능 프렌즈 스피커 노래 재생 후기, 클로바 음악 노래 명령어 활용기 https://blog.naver.com/twinklejrjr?Redirect=Log&logNo=221131258916
네이버 인공지능 스피커 귀여운"브라운"(클로바) :> https://blog.naver.com/solji_?Redirect=Log&logNo=221131320377
네이버 클로바 라인프렌즈 인공지능 스피커 개봉기! https://blog.naver.

인공지능 스피커 프렌즈 브라운 클로바 : 네이버뮤직 https://blog.naver.com/cute_bbotty?Redirect=Log&logNo=221134503850
네이버 클로바 / 라인프렌즈 스피커 돈주고 산 솔직 후기!! https://blog.naver.com/qpsxm63?Redirect=Log&logNo=221134618294
프렌즈 스피커 브라운 개봉기 / 네이버 인공지능 스피커 / 클로바 Clova AI 스피커 https://happydds290.blog.me/221134120954
 https://blog.naver.com/bluebusiness01?Redirect=Log&logNo=221134464122
 https://blog.naver.com/yimdahee__?Redirect=Log&logNo=221134560459
1
네이버 클로바 : 라인프렌즈 인공지능 스피커 연동방법 https://blog.naver.com/purity_k?Redirect=Log&logNo=221135464776
네이버 클로바 AI 스피커: 프렌즈 브라운 개봉기+리뷰 https://blog.naver.com/p_radn?Redirect=Log&logNo=221134982698
클로바 인공지능 스피커 프렌즈 브라운 https://wjd949494.blog.me/221135597002
내 브라운 인공지능스피커를 소개해요 https://blog.naver.com/helloagain92?Redirect=Log&logNo=221135579418
인공지능스피커(feat.블루투스) https://joheey2.blog.me/221135698721
네이버 클로바 라인프렌즈 브라운 인공지능 블루투스 스피커 https://blog.naver.com/syol07?Redirect=Log&logNo=221135618943
네이버 인공지능 스피커 프렌즈 일주일 사용 후기 https://blog.naver.com/sukhee4994?Redirect=Log&logNo=22113545389

CLOVA AI SPEAKER X FRIENDS SALLY 네이버 인공지능 스피커 당첨! https://blog.naver.com/dunnowhat?Redirect=Log&logNo=221142976824
인공지능 스피커 네이버 프렌즈 (clova) -똑똑한 브라운 https://blog.naver.com/h-y-e?Redirect=Log&logNo=221143190839
라인 인공지능 스피커 '프렌즈' 개봉기 (브라운프렌즈/인공지능스피커/라인스피커/프렌즈스피커) https://hanhee112.blog.me/221142978298
 https://blog.naver.com/jjw408?Redirect=Log&logNo=221142590016
 https://blog.naver.com/damiani_37?Redirect=Log&logNo=221143016875
 https://cdmanii.com/6266?category=423952
2
인공지능 스피커 네이버 프렌즈 (clova) -똑똑한 브라운 https://blog.naver.com/h-y-e?Redirect=Log&logNo=221143190839
라인 인공지능 스피커 '프렌즈' 개봉기 (브라운프렌즈/인공지능스피커/라인스피커/프렌즈스피커) https://hanhee112.blog.me/221142978298
 https://blog.naver.com/jjw408?Redirect=Log&logNo=221142590016
 https://blog.naver.com/damiani_37?Redirect=Log&logNo=221143016875
 https://cdmanii.com/6266?category=423952
1
인공지능 스피커 프렌즈 브라운 음질 및 실사용 동영상 https://feena74.blog.me/221143553820
네이버 인공지능 스피커 프렌즈-샐리스피커 https://blog.naver.com/oohazz?Redirect=Log&logNo=221143211703
네이버 클로바 스피커, 

...

LG TV 인공지능 ThinQ! 더 편리하고 매력적인 라이프 스타일을 만들다! https://jayonce1004.blog.me/221456945837
 https://blog.naver.com/ha_tak?Redirect=Log&logNo=221457292972
 https://blog.naver.com/jgg8dv42eky?Redirect=Log&logNo=221457385756
2
LG TV 인공지능 ThinQ! 더 편리하고 매력적인 라이프 스타일을 만들다! https://jayonce1004.blog.me/221456945837
 https://blog.naver.com/ha_tak?Redirect=Log&logNo=221457292972
 https://blog.naver.com/jgg8dv42eky?Redirect=Log&logNo=221457385756
1
집들이 선물 추천 ::네이버 클로바 미니언즈:: https://blog.naver.com/gmltn446?Redirect=Log&logNo=221457649228
 https://blog.naver.com/consumer_quotient?Redirect=Log&logNo=221457604533
2
 https://blog.naver.com/consumer_quotient?Redirect=Log&logNo=221457604533
1
SK 인공지능 스피커 누구 NUGU 캔들 박스 개봉기 https://blog.naver.com/hjhch7?Redirect=Log&logNo=221458135561
2
SK 인공지능 스피커 누구 NUGU 캔들 박스 개봉기 https://blog.naver.com/hjhch7?Redirect=Log&logNo=221458135561
1
 https://blog.naver.com/mnmp_look?Redirect=Log&logNo=221458910601
번개장터 네이버클로바(미니언즈) https://blog.naver.com/tat880?Redirect=Log&logNo=

식빵에 그림과 메시지를 남길 수 있는 스마트토스터 - Toasteriod! https://blog.naver.com/svcbox?Redirect=Log&logNo=221471070275
 https://blog.naver.com/dkfma314?Redirect=Log&logNo=221471245202
 https://blog.naver.com/mrsbs1267?Redirect=Log&logNo=221471277022
1
번개장터 네이버 클로바 블루투스 스피커 https://blog.naver.com/qkqanrm111?Redirect=Log&logNo=221472115820
 https://blog.naver.com/howicontinue?Redirect=Log&logNo=221472093930
 https://blog.naver.com/clova_ai?Redirect=Log&logNo=221471587296
2
 https://blog.naver.com/howicontinue?Redirect=Log&logNo=221472093930
 https://blog.naver.com/clova_ai?Redirect=Log&logNo=221471587296
1
[제주도 마술하는곰] 필립스 휴 3.0 스타터킷, 스마트 스피커 미니언즈, 스트립 플러스 리뷰 2편! : 스마트 스피커 미니언즈 설정, 스트립 플러스 설치 https://blog.naver.com/magicminds?Redirect=Log&logNo=221472921798
2
[제주도 마술하는곰] 필립스 휴 3.0 스타터킷, 스마트 스피커 미니언즈, 스트립 플러스 리뷰 2편! : 스마트 스피커 미니언즈 설정, 스트립 플러스 설치 https://blog.naver.com/magicminds?Redirect=Log&logNo=221472921798
1
1
네이버 클로바 미니언즈 AI 스피커! 블루투스 스피커~ 영쿡의 결혼 선물 https://blog.naver.com/sjh6080?Redirect=Log&logNo=

■[LG전자 퓨리케어 공기청정기 가정용 AS128VWA 38.9㎡ 최신형 구매후기] https://blog.naver.com/ljslol?Redirect=Log&logNo=221484473784
 https://sambolove.blog.me/221484839652
2
브리츠 블루투스 이어폰 Ξ03-10:12Ξ 쿠팡최저가 я 추천상품я https://blog.naver.com/chajungsub24?Redirect=Log&logNo=221484616777
번개장터 미니언즈 네이버 클로바 https://blog.naver.com/tlqkfzm123?Redirect=Log&logNo=221485007338
■[LG전자 퓨리케어 공기청정기 가정용 AS128VWA 38.9㎡ 최신형 구매후기] https://blog.naver.com/ljslol?Redirect=Log&logNo=221484473784
 https://sambolove.blog.me/221484839652
1
 http://acrmedia.co.kr/221485765290
[LG TV 신제품 발표회] AI가 차별화 포인트? 인공지능을 내세운 LG 올레드 TV와 슈퍼울트라 HD TV... https://its.tistory.com/2470821
2
[LG TV 신제품 발표회] AI가 차별화 포인트? 인공지능을 내세운 LG 올레드 TV와 슈퍼울트라 HD TV... https://its.tistory.com/2470821
1
LG유플러스 네이버 클로바프렌즈 미니브라운 초소형 블루투스 스피커클로바! 신곡들려줘! https://blog.naver.com/tiffani48?Redirect=Log&logNo=221486801657
네콘 IoT 제품 설치 및 AI 스피커 연동 방법 https://blog.naver.com/digi0103?Redirect=Log&logNo=221486501760
LG유플러스, 원주 ‘에이스 더블유밸리’ 773세대에 IoT 서비스 https://blog.naver.com/allenhan

번개장터 미니언즈 블루투스 스피커 https://blog.naver.com/wlgpwltn0903?Redirect=Log&logNo=221499619999
 https://blog.naver.com/hyoung7371?Redirect=Log&logNo=221499451317
LG유플러스, 네이버와 손잡고…인수예정 ‘CJ헬로’는 카카오와 손잡아(증권일보.2019.03.28) https://blog.naver.com/enewstock?Redirect=Log&logNo=221499625909
 http://lillaking.com/221499596664
 https://xaxo.tistory.com/2154
샤오미 미에어2 :: 가성비갑이라는 샤오미 공기청정기 미에어2 한달후기 https://blog.naver.com/sel__2?Redirect=Log&logNo=221499523913
엘지유플러스 : 아이들나라, 넷플릭스, U+tv브라보라이프까지! 온가족 다 함께 즐기는 인생TV https://blog.naver.com/lsonl?Redirect=Log&logNo=221499957792
2 0 1 9 0 3 월 일 상 :: 별거없는데요즘엄청사진찍는주간/내 발목인데 진짜 별로다/ 샤오미홈키트리뷰/26만원에초연해보자/안갈병원추가/에어팟2사기극/요즘카페음료양미쳤음 https://newjjyi.blog.me/221499808953
2
번개장터 미니언즈 블루투스 스피커 https://blog.naver.com/wlgpwltn0903?Redirect=Log&logNo=221499619999
 https://blog.naver.com/hyoung7371?Redirect=Log&logNo=221499451317
LG유플러스, 네이버와 손잡고…인수예정 ‘CJ헬로’는 카카오와 손잡아(증권일보.2019.03.28) https://blog.naver.com/enewstock?Redirect=Log&logNo=221499625909
 http://lillaking.com/22149959

 https://blog.naver.com/pretty-namhee-?Redirect=Log&logNo=221516273861
 https://blog.naver.com/yjjone21?Redirect=Log&logNo=221516290555
2
아들과 딸 북클럽, 인공지능 북클럽 LG클로이 정말 똑똑하네요~ 헤이클로이! https://blog.naver.com/pusanjunga?Redirect=Log&logNo=221515715235
 https://blog.naver.com/rlacl2848?Redirect=Log&logNo=221515832602
 https://blog.naver.com/jjokok27?Redirect=Log&logNo=221515513541
네이버 클로바] 어플 다운로드 후 와이파이 연결했어요^^ https://blog.naver.com/ppp3999?Redirect=Log&logNo=221516097150
 https://blog.naver.com/pretty-namhee-?Redirect=Log&logNo=221516273861
 https://blog.naver.com/yjjone21?Redirect=Log&logNo=221516290555
1
유기지능 스튜디오 오디오영화 사피엔스 짝짓기 현실적이면서 재밌게 들었어요. https://blog.naver.com/du851108?Redirect=Log&logNo=221516771746
 https://blog.naver.com/barohills?Redirect=Log&logNo=221516608465
 https://blog.naver.com/dlvmfflq1?Redirect=Log&logNo=221516335089
[엘지 ThinQ 허브,게이트웨이/씽큐허브에,숙박,모텔/원룸/오피스텔/빌라 LG 특판/납품용] LG전자 엘지전자 B2B 기업특판팀/납품팀에서 안내하여 드립니다 http://iamits.co.kr/221510856983
 http://xn--v92bi4lpnn3qd.kr/

In [89]:
total_url

['https://blog.naver.com/bonamy?Redirect=Log&logNo=221087693369',
 'https://outia.blog.me/221087617932',
 'https://byline.network/2017/09/7-3/',
 'https://blog.naver.com/withcult?Redirect=Log&logNo=221092527285',
 'https://blog.naver.com/dndyddka?Redirect=Log&logNo=221094640305',
 'https://clay1987.blog.me/221094020655',
 'https://blog.naver.com/clova_ai?Redirect=Log&logNo=221095185987',
 'https://blog.naver.com/honest72?Redirect=Log&logNo=221097077052',
 'http://blog.daum.net/0116010989/15895375',
 'https://blog.naver.com/jcs203?Redirect=Log&logNo=221098685975',
 'http://asnote.net/221099169085',
 'https://blog.naver.com/ejo5hugtp?Redirect=Log&logNo=221099706732',
 'https://blog.naver.com/idolpia?Redirect=Log&logNo=221100021432',
 'https://byline.network/2017/09/19-7/',
 'https://how-r-u.tistory.com/298',
 'https://blog.naver.com/wogur0217?Redirect=Log&logNo=221102322715',
 'https://blog.naver.com/etlandking?Redirect=Log&logNo=221102185156',
 'https://blog.naver.com/kanoc28?Redirect=L

## Get real_url, title, datetime, text

In [94]:
with open('네이버_클로바_스피커.txt', 'w', encoding = 'utf8') as f:
    title_list = []
    publish_datetime_list = []
    text_list = []
    
    for full_url in tqdm_notebook(total_url):
        
        # 실제 데이터가 들어있는 url 추출
        real_url = get_realdata_url_naverblog(full_url)
        r = requests.get(real_url)
        soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

        # 블로그 포스트 제목 추출
        title = get_title_naverblog(soup)
        title_list.append(title)

        # 블로그 작성일 추출
        publish_datetime = det_publish_datetime_naverblog(soup)
        publish_datetime_list.append(publish_datetime)
        
        # 블로그 텍스트 추출
        texts = get_texts_naverblog(soup)
        text_list.append(texts)

#         print('url: ', full_url)
#         print('real_url: ', real_url, '\n')
#         print('post title: ', title, '\n')
#         print('publish date time: ', publish_datetime, '\n')
#         print("texts: ", texts)
#         print("\n-----------------------------------\n")
        
        #content = ' '.join(texts)
        f.write(texts+'\n') #결과를 text 파일에 저장합니다.

        time.sleep(0.1)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [403

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [200]>
<Response [200

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [110]:
title_list[:30]

["블루투스 라디오 알람시계 'iLuv 모닝콜2' 리뷰 & 어플추천",
 'WAVE 리뷰를 하려다가 QA?',
 'AI, 클라우드 플랫폼이라는 전장에서 모두가 만난다 - Byline Network',
 '[위드컬처/트렌드] 미래의 거주환경, 스마트 홈이 다가온다',
 'AI 스피커 카카오미니  18일부터 예약판매\u3000\u3000\u3000\u3000한글그림기적',
 '170911. 뉴스 모아보기 : 네이버의 AI와 클라우드 플랫폼, 파파고, 클로바 서비스',
 '인공지능 스피커 WAVE 파격 프로모션 소식 (9/14 커밍순)',
 'AI 짝짓기 계절···카카오아이·빅스비 뭉친다 [서울경제]',
 'NA',
 '“목소리로 메일 보내고 송금까지…앱은 사라질 것”',
 'AI 스피커 인공지능 플랫폼 동맹: 삼성-카카오, 네이버-LG, 그리고 애플 구글의 행보',
 '네이버 인공지능 스피커 사용법 , 클로바 설치!',
 '네이버 웨이브와  파트너 제휴를 맺은 기업과 브랜드를 통해 알수 있는 예상 서비스 (클로바,Clova)',
 '심스키의 일기, 샐리와 보낸 하루 - Byline Network',
 'NA',
 '똑똑한 인공지능 AI 스피커 네이버 wave 스피커 1달 사용후기[영상포함]',
 '네이버, 카카오 AI 스피커 최강자전! 네이버 웨이브 vs 카카오 미니',
 '네이버 인공지능 스피커 : Wave',
 '네이버 인공지능 스피커 웨이브 wave',
 '사진으로 만나는 ‘네이버 웨이브’',
 'AI 스피커 종류와 플랫폼: 네이버, 카카오미니 etc.',
 'Clova News Letter Vol.1| 한가위 연휴 Clova 활용 꿀팁',
 "카카오가 그리는 AI 생태계 '청사진'",
 "스마트한 AI 음성비서 대전...'누가누가 잘하나'",
 "'한국어 적응' 구글어시스턴트 써보니...맥락 이해↑",
 '조회수 순위 목록 2017년 10월 02일 기준  ',
 '네이버 웨이브-개봉기',
 '#지식산업 004. 넌 내말을 얼마나 이해하니? : 음성인식 서비

In [114]:
print(text_list[100])

라인 인공지능 스피커를 샀다!근데 내가 생각하는 인공지능은 아닌가봐...사무실에서 시연해보다가 왜이렇게 못알아듣냐고...그래서 인공지능이라고 다 똑똑하진 않을거라고 항변.네이버 인공지능 스피커 wave가 라인캐릭터랑 콜라보로 스피커를 출시 했고,네이버 뮤직 1년 이용권과 패키지로 선택할 수 있었다.샐리와 브라운중에 심각하게 고민하다가 때 덜탈것 같아서 브라운으로 선택했는데....샐리가 귀여워서 좀 후회했었다.
그 분이 왔다!!!네이버 뮤직 1년 이용권을 결제하면 인공지능 스피커를 주고,교환,환불 시 감가상각으로 인해 까다롭다.난 원래 환불이나 교환은 앵간해서는 귀찮아서 잘 안하는 타입이라 개의치 않고 구입.사실 뭔지도 모르고 샀다.사은품,잿밥,경품,별책부록들을 워낙 애정하는지라 어차피 노래는 멜론으로 듣고 있는데 네이버 뮤직으로 갈아타고 스피커 받자라는 생각으로 질렀다.(블루투스 스피커를 세개나 사냐며...사무실 직원으로부터 허랑방탕하다는 놀림도 받음..ㅋ)
나름 합리적인 소비였어..
브라운의 고운 자태사무실로 택배를 받아서 몰래 뜯어보기 시작
은박씰이 붙어있는데 뗐다 붙였다 하면 금방 티난다.
요래요래 자국이 남는다.
너무 귀여워♥
가이드북과 충전기는 플라스틱 틀 아래 들어있다.충전케이블은 요새 새로 나온 C형 케이블이다.그 둥그스름한...
하단에 전원버튼이 있는데 조용한 사무실에서 눌러봤다가 갱장히 민망한 상황이 발생됐다.아주 우렁찬 목소리로 인사를 하는데 몰래 택배를 뜯어보려던 나의 의도와는 다르게 만천하에 공개.(우리 사무실은 두칸으로 나뉘어있는데 내자리는 어려운 분들이 계셔서 행동거지를 조심하게 된다.ㅠㅠ)엄청 당황해서 브라운 들고 달려나갔다..하하;;
밑에 하얀띠 부분에 불이 들어온다.네이버 클로바 앱을 다운받고 연동하고 "클로바"를 불러서 브라운이 인식을 하면 하얀띠에 녹색불이 들어온다.
어떤걸 물어야할지 예시가 나와있다.와이파이가 설치되어 있어야 연동이 되고,업데이트가 완료되기 전까지 충전기를 전원에서 분리하면 안된다.브라운은 무광인데 기름때 티

## dataframe.to_csv

In [95]:
import pandas as pd
df = pd.DataFrame(data={'title': title_list, 
                        'publish_datetime': publish_datetime_list,
                        'text': text_list, 
                        'url': total_url})

In [96]:
#df = df[['title', 'publish_datetime', 'text', 'url']]
print(df.shape)
df.head()

(2224, 4)


,title,publish_datetime,text,url
0,블루투스 라디오 알람시계 'iLuv 모닝콜2' 리뷰 & 어플추천,2017-09-02 11:43:00,"침대 옆 공간은 좁지만 놓아야 할 것들이 꽤 많아서 늘 비좁은데, 침대맡에 하나 두...",https://blog.naver.com/bonamy?Redirect=Log&log...
1,WAVE 리뷰를 하려다가 QA?,2017-09-02 08:50:00,WAVE 를 입수한 김에 큰 맘먹고 리뷰를 한 번 써보려다가 결국 근사한 리뷰작성 ...,https://outia.blog.me/221087617932
2,"AI, 클라우드 플랫폼이라는 전장에서 모두가 만난다 - Byline Network",NA,NA,https://byline.network/2017/09/7-3/
3,"[위드컬처/트렌드] 미래의 거주환경, 스마트 홈이 다가온다",2017-09-08 19:04:00,"[위드컬처/트렌드] 미래의 거주환경, 스마트 홈이 다가온다\n4차 산업 혁명 시대,...",https://blog.naver.com/withcult?Redirect=Log&l...
4,AI 스피커 카카오미니 18일부터 예약판매 한글그림기적,2017-09-11 22:40:00,"\n카카오 미니, 예약 판매 가격은?...네이버 클로바 경쟁한국경제TV 8시간 전...",https://blog.naver.com/dndyddka?Redirect=Log&l...


In [119]:
df.to_csv('./naver_clova_blog.csv',sep=',',encoding='UTF-16')

In [121]:
pd.read_csv('./naver_clova_blog.csv', encoding='UTF-16',index_col=0)

,title,publish_datetime,text,url
0,블루투스 라디오 알람시계 'iLuv 모닝콜2' 리뷰 & 어플추천,2017-09-02 11:43:00,"침대 옆 공간은 좁지만 놓아야 할 것들이 꽤 많아서 늘 비좁은데, 침대맡에 하나 두...",https://blog.naver.com/bonamy?Redirect=Log&log...
1,WAVE 리뷰를 하려다가 QA?,2017-09-02 08:50:00,WAVE 를 입수한 김에 큰 맘먹고 리뷰를 한 번 써보려다가 결국 근사한 리뷰작성 ...,https://outia.blog.me/221087617932
2,"AI, 클라우드 플랫폼이라는 전장에서 모두가 만난다 - Byline Network",NaN,NaN,https://byline.network/2017/09/7-3/
3,"[위드컬처/트렌드] 미래의 거주환경, 스마트 홈이 다가온다",2017-09-08 19:04:00,"[위드컬처/트렌드] 미래의 거주환경, 스마트 홈이 다가온다\n4차 산업 혁명 시대,...",https://blog.naver.com/withcult?Redirect=Log&l...
4,AI 스피커 카카오미니 18일부터 예약판매 한글그림기적,2017-09-11 22:40:00,"\n카카오 미니, 예약 판매 가격은?...네이버 클로바 경쟁한국경제TV 8시간 전...",https://blog.naver.com/dndyddka?Redirect=Log&l...
5,"170911. 뉴스 모아보기 : 네이버의 AI와 클라우드 플랫폼, 파파고, 클로바 서비스",2017-09-11 09:47:00,https://byline.network/2017/06/30/\n네이버가 인공지능 ...,https://clay1987.blog.me/221094020655
6,인공지능 스피커 WAVE 파격 프로모션 소식 (9/14 커밍순),2017-09-12 15:59:00,안녕하세요막내 개발자 빠암~ 입니다네이버 클로바 플랫폼의 인공지능 스피커! WAVE...,https://blog.naver.com/clova_ai?Redirect=Log&l...
7,AI 짝짓기 계절···카카오아이·빅스비 뭉친다 [서울경제],2017-09-14 22:03:00,"\n카카오, 해외 네트워크 강화 기대제네시스 G70에도 음성인식 탑재네이버 클로바,...",https://blog.naver.com/honest72?Redirect=Log&l...
8,NaN,NaN,NaN,http://blog.daum.net/0116010989/15895375
9,“목소리로 메일 보내고 송금까지…앱은 사라질 것”,2017-09-17 11:07:00,\n“목소리로 메일 보내고 송금까지…앱은 사라질 것”]시대의 패권을 놓고 기술 경쟁...,https://blog.naver.com/jcs203?Redirect=Log&log...
